# Biopython
*[Reference online](https://biopython.org)*



In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 10.6 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 52.1 MB/s eta 0:00:00 MB/s eta 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# Classi base: `Seq`

Definisce una sequenza generica, equipaggiata con metodi orientati alle sequenze genetiche.

In [2]:
from Bio.Seq import Seq


sequence_str = "AUGAACTGGTAUUA"
sequence = Seq(sequence_str)

print(f"Sequence: {sequence}")
print(f"Sequence complement: {sequence.complement()}")
print(f"Sequence reverse complement: {sequence.reverse_complement()}")

print(f"DNA to RNA: {sequence.transcribe()}")
print(f"RNA to DNA: {sequence.transcribe().back_transcribe()}")

Sequence: AUGAACTGGTAUUA
Sequence complement: TACTTGACCATAAT
Sequence reverse complement: TAATACCAGTTCAT
DNA to RNA: AUGAACUGGUAUUA
RNA to DNA: ATGAACTGGTATTA


In [3]:
sequence[3], sequence[:3], sequence[:3] + sequence[5:]

('A', Seq('AUG'), Seq('AUGCTGGTAUUA'))

### Ricerca sottostringhe

In [4]:
print(sequence.index("CTG"))

5


In [5]:
print(sequence.count("CTG"))

1


### Sintetizzazione proteine

Alcune sequenze di DNA sintetizzano proteine mediante traduzioni successive `DNA > mRNA > catena amminoacidi > proteina`. Questi son mediati da diverse tabelle di traduzione che mappano sottosequenze in amminoacidi.

Possibili tabelle disponibili su [NCBI](https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi).

In [6]:
sequence.translate(table="Vertebrate Mitochondrial")

/home/ms/.virtualenvs/biotech/lib/python3.12/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Seq('MNWY')

In [7]:
from Bio.Data import CodonTable


table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"]

print(table)

Table 2 Vertebrate Mitochondrial, SGC1

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA W   | A
T | TTG L   | TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L   | CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I(s)| ACT T   | AAT N   | AGT S   | T
A | ATC I(s)| ACC T   | AAC N   | AGC S   | C
A | ATA M(s)| ACA T   | AAA K   | AGA Stop| A
A | ATG M(s)| ACG T   | AAG K   | AGG Stop| G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V(s)| GCG A   | GAG E   | GGG G   

Queste tabelle possono essere lette come seque:
- prima colonna: prima lettera del codone
- prima riga: seconda lettera del codone
- celle: codone completo

---

# Sequenze... and more: `SeqRecord`

[Reference online](https://biopython.org/wiki/SeqRecord).

La classe `SeqRecord` permette di avere informazioni aggiuntive.

In [8]:
from Bio.SeqRecord import SeqRecord

genome = Seq("ACTGATTCGGACTCTAAGTTA")
organism = SeqRecord(
    genome,
    id="organism_4523145",
    name="lab_rat",
    description="Laboratory rat, taken in Fucecchio."
)

In [9]:
organism.id, organism.name

('organism_4523145', 'lab_rat')

`SeqRecord`s mantengono le funzioni di `Seq`.

In [10]:
organism[:5]

SeqRecord(seq=Seq('ACTGA'), id='organism_4523145', name='lab_rat', description='Laboratory rat, taken in Fucecchio.', dbxrefs=[])

---

# Sequenze su disco: `FASTA`

Sequenze genetiche e di proteine sono spesso salvate in formati specifi, e.g., `FASTA`.

Formato
```
>{descrizione}
{sequenza}
```

E.g.,
```
>MCHU - Calmodulin - Human, rabbit, bovine, rat, and chicken
MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAELQDMINEVDADGNGTID
FPEFLTMMARKMKDTDSEEEIREAFRVFDKDGNGYISAAELRHVMTNLGEKLTDEEVDEMIREA
DIDGDGQVNYEEFVQMMTAK*
```

Il formato viene definito regolamentato da NIH: [reference online](https://blast.ncbi.nlm.nih.gov/doc/blast-topics/).
Nella descrizione possiamo inserire diverse informazioni, alcune con degli standard definiti, e.g., per indicare la provenienza di dati ([reference online](https://ncbi.github.io/cxx-toolkit/pages/ch_demo#ch_demo.id1_fetch.html_ref_fasta)).

In [12]:
from Bio import SeqIO


# fonte: https://www.rcsb.org/downloads/fasta
sequences = list(SeqIO.parse("data/pdb_seqres.fasta", "fasta"))

In [13]:
sequences[0]

SeqRecord(seq=Seq('CCGGCGCCGG'), id='100d_A', name='100d_A', description="100d_A mol:na length:10  DNA/RNA (5'-R(*CP*)-D(*CP*GP*GP*CP*GP*CP*CP*GP*)-R(*G)-3')", dbxrefs=[])

# Sequenze su disco: `Genbank`

Formato e database manutenuto da [NIH](https://www.ncbi.nlm.nih.gov/genbank/), e che unisce sequenze genetiche da database Europei e Giapponesi.

In [15]:
# fonte: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8359044/
orchids = list(SeqIO.parse("data/ls_orchid.gbk", "genbank"))

In [16]:
print(
    # orchids[0].id,
    # orchids[0].name,
    # orchids[0].description,
    # orchids[0].seq,
    orchids[0].annotations
)

{'molecule_type': 'DNA', 'topology': 'linear', 'data_file_division': 'PLN', 'date': '30-NOV-2006', 'accessions': ['Z78533'], 'sequence_version': 1, 'gi': '2765658', 'keywords': ['5.8S ribosomal RNA', '5.8S rRNA gene', 'internal transcribed spacer', 'ITS1', 'ITS2'], 'source': 'Cypripedium irapeanum', 'organism': 'Cypripedium irapeanum', 'taxonomy': ['Eukaryota', 'Viridiplantae', 'Streptophyta', 'Embryophyta', 'Tracheophyta', 'Spermatophyta', 'Magnoliophyta', 'Liliopsida', 'Asparagales', 'Orchidaceae', 'Cypripedioideae', 'Cypripedium'], 'references': [Reference(title='Phylogenetics of the slipper orchids (Cypripedioideae: Orchidaceae): nuclear rDNA ITS sequences', ...), Reference(title='Direct Submission', ...)]}


# Salvare sequenze su disco

In [ ]:
SeqIO.write(orchids[:1], handle="orchids.faa", format="fasta")

# Sequenze su disco... remoto: `Entrez`

NIH offre accesso a diversi database pubblici [direttamente online](https://www.ncbi.nlm.nih.gov)! `biopython` ci permette di avere accesso diretto.

In [17]:
from Bio import Entrez
from Bio import SeqIO


with Entrez.efetch(db="protein", rettype="gb", retmode="text", id="157427902") as handle:
    tyrosine = SeqIO.read(handle, "gb")
print(tyrosine)

/home/ms/.virtualenvs/biotech/lib/python3.12/site-packages/Bio/Entrez/__init__.py:734: UserWarning: 
            Email address is not specified.

            To make use of NCBI's E-utilities, NCBI requires you to specify your
            email address with each request.  As an example, if your email address
            is A.N.Other@example.com, you can specify it as follows:
               from Bio import Entrez
               Entrez.email = 'A.N.Other@example.com'
            In case of excessive usage of the E-utilities, NCBI will attempt to contact
            a user at the email address provided before blocking access to the
            E-utilities.
  warnings.warn(


ID: NP_001098858.1
Name: NP_001098858
Description: tyrosine-protein kinase ITK/TSK [Bos taurus]
Number of features: 18
/topology=linear
/data_file_division=MAM
/date=03-FEB-2022
/accessions=['NP_001098858', 'XP_600591']
/sequence_version=1
/db_source=REFSEQ: accession NM_001105388.1
/keywords=['RefSeq']
/source=Bos taurus (domestic cattle)
/organism=Bos taurus
/taxonomy=['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Laurasiatheria', 'Artiodactyla', 'Ruminantia', 'Pecora', 'Bovidae', 'Bovinae', 'Bos']
/references=[Reference(title='A positional candidate gene association analysis of susceptibility to paratuberculosis on bovine chromosome 7', ...), Reference(title='A whole-genome assembly of the domestic cow, Bos taurus', ...)]
/comment=PROVISIONAL REFSEQ: This record has not yet been subject to final
NCBI review. The reference sequence was derived from BC153237.1.
On Sep 25, 2007 this sequence version replaced XP_600591.3.
/structu

In [18]:
with Entrez.efetch(db="nuccore", rettype="gb", retmode="text", id="X14848.1") as handle:
    r = SeqIO.read(handle, "gb")
print(r)

ID: X14848.1
Name: X14848
Description: Rattus norvegicus mitochondrial genome
Number of features: 40
/molecule_type=DNA
/topology=circular
/data_file_division=ROD
/date=26-JUL-2016
/accessions=['X14848', 'V01574']
/sequence_version=1
/keywords=['12S ribosomal RNA', '16S ribosomal RNA', 'ATPase', 'ATPase subunit 6', 'ATPase subunit 8', 'complete genome', 'cytochrome b', 'cytochrome c oxidase', 'cytochrome c oxidase subunit I', 'cytochrome c oxidase subunit II', 'cytochrome c oxidase subunit III', 'NADH dehydrogenase', 'NADH dehydrogenase subunit 1', 'NADH dehydrogenase subunit 2', 'NADH dehydrogenase subunit 3', 'NADH dehydrogenase subunit 4', 'NADH dehydrogenase subunit 4L', 'NADH dehydrogenase subunit 5', 'NADH dehydrogenase subunit 6', 'origin of replication', 'ribosomal RNA', 'transfer RNA', 'transfer RNA-Ala', 'transfer RNA-Arg', 'transfer RNA-Asn', 'transfer RNA-Asp', 'transfer RNA-Cys', 'transfer RNA-Gln', 'transfer RNA-Glu', 'transfer RNA-Gly', 'transfer RNA-His', 'transfer RNA-

---

# Sequenze e allineamento
*Sequence alignment* ci permette di allineare due o piu' sequenze in modo da farle combaciare quanto piu' possibile.

```
s: A C C T G A
t: A T G A
```
possono essere allineate in
```
s: A C C T G A
t: A - - T G A
```
dove `-` indica elementi mancanti. L'allineamento e' utile per, e.g.,
- trovare sottoaree del genoma comune
- ricostruire relazioni filogenetiche tra genomi
- ricostruire sequenze parziali

Risolvere sequence alignment e' un problema **complesso**, a volte **approssimato**, e che puo' accettare **piu' soluzioni**.

---

# Sequenze e similarita'
Oltre a allineare sequenze, possiamo computarne la similarita': quanto e' il dato genoma `s` simile a un dato genoma `t`?

Spesso utilizzato per fare un matching tra un nuovo genoma e una banca dati di genomi. Come per l'allineamento, gli algoritmi possono essere approssimati, e le soluzioni non uniche. Useremo la suite di algoritmi BLAST (Basic Local Alignment Search Tool), che propone diversi algoritmi:
- blastn
- blastp
- blastx
- tblast
- tblastx

In [ ]:
from Bio.Blast.NCBIWWW import qblast

result_stream = NCBIWWW.qblast("blastn", "nt", "8332116")
data = SearchIO.read(result_stream, "blast-xml")
print(data),

I vari match sono `Hit`, ognuno con diverse proprieta':
- score di similarita': `evalue`
- score di similarita': `bitscore`
- allineamenti: `aln`

In [ ]:
best_match = data[0]

In [ ]:
best_match[0].evalue, best_match[0].bitscore

In [ ]:
alignment = best_match[0].aln
len(alignment)

In [ ]:
alignment[0]

In [ ]:
a = alignment[0]

In [ ]:
a.annotations
# id
# annotations
# description
# features